## ImageGenerator 사용하기
주의: 이 코드는 코랩에서 돌려야 함(지피유 메모리 에러남)

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(# Your Code):
  # Your Code

callbacks = myCallback()
    

# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
])

from tensorflow.keras.optimizers import RMSprop

model.compile(# Your Code Here #)
    

# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = # Your Code Here

train_generator = train_datagen.flow_from_directory(
        # Your Code Here)

# Expected output: 'Found 80 images belonging to 2 classes'
    

# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      # Your Code Here)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

SyntaxError: invalid syntax (<ipython-input-1-9b6a852ed885>, line 21)

## Answer

In [2]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACC = 0.999 # 이렇게 하니까 C코드 생각난당

### Load Data
~~다운로드 경로 바꾸고 싶은데...~~
* 터미널에서 wget -h 하면 무지 많은 옵션이 나옴
* wget -O 원하는URL 다운받을 경로
    * (대문자 O임)

허걱 왜 /tmp에다가 저장하는지 알았다!!! 리눅스 폴더구조 공부하면 됨!!!
* tmp
 - 실행 중인 프로세스들의 임시 파일들을 저장
 - **종료 시 이 위치에 있는 파일들은 모두 삭제됨**
 - 그래서 이름이 tmp이구나..
* [리눅스 폴더구조 및 용도 잘 정리해둔 블로그 참고](https://blackpigstudio.tistory.com/entry/%EB%A6%AC%EB%88%85%EC%8A%A4-%ED%8F%B4%EB%8D%94%EA%B5%AC%EC%A1%B0-%EB%B0%8F-%EC%9A%A9%EB%8F%84)

In [3]:
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" -O "/tmp/happy-or-sad.zip"

--2021-01-13 21:00:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.80, 34.64.4.16, 2404:f340:10:1802::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  8.12MB/s    in 0.3s    

2021-01-13 21:00:04 (8.12 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [4]:
zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("./data/happy-or-sad")
zip_ref.close()

### Callback 함수 정의
이따가 쓸거지만 지금 정의먼저 해두자

In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>DESIRED_ACC):
            print("\n\nI did it!!")
            self.model.stop_training = True

In [6]:
callbacks = myCallback()

### Data 보기

#### (1) 데이터 개수 추출

In [11]:
train_dir_happy = os.path.join('./data/happy-or-sad/happy')
train_dir_sad = os.path.join('./data/happy-or-sad/sad')

In [14]:
train_name_happy = os.listdir(train_dir_happy)
train_name_sad = os.listdir(train_dir_sad)

In [16]:
print(len(train_name_happy))
print(len(train_name_happy))

40
40


엥 데이터 무지 쪼금이네 이정도면 GPU 로컬로 돌릴 수 있겠당

#### (2) 데이터 사이즈 추출

In [17]:
train_name_happy[0]

'happy2-15.png'

In [18]:
test_name = train_name_happy[0]
test_path = os.path.join(train_dir_happy, test_name)
test_path

'./data/happy-or-sad/happy/happy2-15.png'

In [21]:
from PIL import Image

test_img = Image.open(test_path)
test_img.show()

In [22]:
test_img.size

(150, 150)

### Modeling

In [34]:
model = tf.keras.Sequential([
    # 이제 convolutional layer 끼얹기
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # 일단 DNN 구조먼저
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') #binary classification
])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 16)        4624      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 128)       18560     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36992)            

In [36]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

* 참고 : RMSprop의 default learning rate는 0.001이다.

### ImageGenerator

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
train_datagen = ImageDataGenerator(1./255)

In [29]:
train_generator = train_datagen.flow_from_directory(
    './data/happy-or-sad',
    target_size=(150,150),
    batch_size=8,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [37]:
history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=20, # 20번 돌리고 0.999 이상 나오면 멈추도록
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[80,32,74,74] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/max_pooling2d_3/MaxPool (defined at <ipython-input-37-8d924361078c>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2227]

Function call stack:
train_function


이렇게 데이터가 적고 크기가 작고 배치사이즈가 작은데도 메모리 에러가 난단말야??ㅠㅠㅠㅠㅠ

모델 파라미터를 작게해도 그렇네....